In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("sample.pdf")
pdf_document=loader.load()
pdf_document

[Document(page_content='Wi-SensiNet: Through-Wall Human Activity\nRecognition Based on WiFi Sensing\nFu Xiaoyi\xa0\nSouthwest University\nWang Chenlu\xa0\nSouthwest University\nLi Shenglin\xa0 \n \nSouthwest University\nResearch Article\nKeywords:  Human activity recognition, Deep learning, WiFi se nsing, Through wall,Channel state\ninformation\nPosted Date:  March 19th, 2024\nDOI:  https://doi.org/10.21203/rs.3.rs-4106293/v1\nLicense:  \uf25e \uf4e7 This work is licensed under a Creative Commons Att ribution 4.0 International License. \xa0\nRead Full License\nAdditional Declarations:  No competing interests reported.', metadata={'source': 'sample.pdf', 'page': 0}),
 Document(page_content='Wi-SensiNet: Through-Wall Human Activity\nRecognition Based on WiFi Sensing\nFu Xiaoyi1, Wang Chenlu1, Li Shenglin1*\n1*College of Artiﬁcial Intelligence, Southwest University, Chongqing,\n400715, China.\n*Corresponding author(s). E-mail(s): lishenglin@swu.edu.cn ;\nContributing authors: swufuxiaoyi@

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
pdf_doc=splitter.split_documents(pdf_document)
pdf_doc


[Document(page_content='Wi-SensiNet: Through-Wall Human Activity\nRecognition Based on WiFi Sensing\nFu Xiaoyi\xa0\nSouthwest University\nWang Chenlu\xa0\nSouthwest University\nLi Shenglin\xa0 \n \nSouthwest University\nResearch Article\nKeywords:  Human activity recognition, Deep learning, WiFi se nsing, Through wall,Channel state\ninformation\nPosted Date:  March 19th, 2024\nDOI:  https://doi.org/10.21203/rs.3.rs-4106293/v1\nLicense:  \uf25e \uf4e7 This work is licensed under a Creative Commons Att ribution 4.0 International License. \xa0\nRead Full License\nAdditional Declarations:  No competing interests reported.', metadata={'source': 'sample.pdf', 'page': 0}),
 Document(page_content='Wi-SensiNet: Through-Wall Human Activity\nRecognition Based on WiFi Sensing\nFu Xiaoyi1, Wang Chenlu1, Li Shenglin1*\n1*College of Artiﬁcial Intelligence, Southwest University, Chongqing,\n400715, China.\n*Corresponding author(s). E-mail(s): lishenglin@swu.edu.cn ;\nContributing authors: swufuxiaoyi@

In [4]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

db=FAISS.from_documents(pdf_doc,OllamaEmbeddings())

### importing llama2 

In [7]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama2")
llm

Ollama()

### Desigining ChatPrompt Templete

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template("""Answer the question only on the basis of context provided. 
                          Take provided context into consideration before providing a answer.
                          <context>{context}</context>
                          question:{input}""")



## Chain
### Create Stuff Document Chain

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)


## Retrievers

In [11]:
retviever=db.as_retriever()
retviever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002256012ACD0>)

### Reteriver chain

In [12]:
from langchain.chains import create_retrieval_chain
retvieval_chain=create_retrieval_chain(retviever,document_chain)


In [14]:
response=retvieval_chain.invoke({"input":"Give the summery of given context"})
response['answer']

"Sure! Here's a summary of the provided context:\n\nThe article discusses the challenges of human activity recognition (HAR) in complex environments using wireless signals. The authors propose a novel approach called Wi-SensiNet, which leverages both convolutional neural networks (CNNs) and bi-directional long short-term memory (BiLSTM) networks to recognize human activities through walls. The proposed method utilizes channel state information (CSI) data to extract and utilize the most representative features within wireless signals. The authors evaluate the performance of Wi-SensiNet on a through-wall CSI dataset comprising seven common activities and achieve an average accuracy of over 99%. The results demonstrate the model's robustness and high accuracy in handling HAR tasks in complex environments, highlighting the potential of CNNs and BiLSTMs working together to enhance performance."